# FX Limitations Demo: HuggingFace ResNet vs Pure ResNet

This notebook demonstrates why HuggingFace ResNet fails with FX symbolic tracing while pure ResNet works perfectly.

## The Issue
HuggingFace ResNet includes input validation that compares tensor shapes at runtime, which breaks FX symbolic tracing.

In [1]:
import traceback

import torch
import torch.fx
import torch.nn as nn
from transformers import ResNetForImageClassification

## 1. Pure ResNet Implementation (FX Compatible)

In [2]:
class SimpleResNet(nn.Module):
    """Pure ResNet implementation without input validation - FX compatible"""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)
        
        # Simple residual block
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 1000)
        
    def forward(self, x):
        # Initial layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Residual block - NO tensor shape validation!
        identity = x
        out = self.conv2(x)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out += identity  # This is fine - just tensor addition
        out = self.relu(out)
        
        # Classification
        out = self.avgpool(out)
        out = out.flatten(1)
        return self.fc(out)

# Test pure ResNet with FX
print("=== Testing Pure ResNet with FX ===")
simple_resnet = SimpleResNet()
dummy_input = torch.randn(1, 3, 224, 224)

try:
    fx_graph = torch.fx.symbolic_trace(simple_resnet)
    print("✅ Pure ResNet: FX tracing SUCCESSFUL!")
    print(f"   FX graph nodes: {len(list(fx_graph.graph.nodes))}")
    
    # Test execution
    output = fx_graph(dummy_input)
    print(f"   Output shape: {output.shape}")
    
except Exception as e:
    print(f"❌ Pure ResNet FX failed: {e}")
    traceback.print_exc()

=== Testing Pure ResNet with FX ===
✅ Pure ResNet: FX tracing SUCCESSFUL!
   FX graph nodes: 16
   Output shape: torch.Size([1, 1000])


## 2. HuggingFace ResNet (FX Incompatible)

In [3]:
# Load HuggingFace ResNet
print("=== Testing HuggingFace ResNet with FX ===")
hf_resnet = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')

try:
    fx_graph = torch.fx.symbolic_trace(hf_resnet)
    print("✅ HuggingFace ResNet: FX tracing successful!")
    
except Exception as e:
    print(f"❌ HuggingFace ResNet FX failed: {e}")
    print("\n📍 The problematic line is in ResNetEmbeddings.forward():")
    print("   num_channels = pixel_values.shape[1]")
    print("   if num_channels != self.num_channels:  # <-- This breaks FX!")

=== Testing HuggingFace ResNet with FX ===
❌ HuggingFace ResNet FX failed: symbolically traced variables cannot be used as inputs to control flow

📍 The problematic line is in ResNetEmbeddings.forward():
   num_channels = pixel_values.shape[1]
   if num_channels != self.num_channels:  # <-- This breaks FX!


## 3. Let's Look at the Exact Problematic Code

In [4]:
import inspect

from transformers.models.resnet.modeling_resnet import ResNetEmbeddings

print("=== HuggingFace ResNetEmbeddings.forward() Source ===")
source = inspect.getsource(ResNetEmbeddings.forward)
lines = source.split('\n')

for i, line in enumerate(lines, 1):
    if 'if num_channels !=' in line:
        print(f"🚨 {i:2}: {line}  <-- PROBLEMATIC LINE!")
    elif 'num_channels = pixel_values.shape[1]' in line:
        print(f"⚠️  {i:2}: {line}  <-- Creates Proxy object")
    else:
        print(f"   {i:2}: {line}")

=== HuggingFace ResNetEmbeddings.forward() Source ===
    1:     def forward(self, pixel_values: Tensor) -> Tensor:
⚠️   2:         num_channels = pixel_values.shape[1]  <-- Creates Proxy object
🚨  3:         if num_channels != self.num_channels:  <-- PROBLEMATIC LINE!
    4:             raise ValueError(
    5:                 "Make sure that the channel dimension of the pixel values match with the one set in the configuration."
    6:             )
    7:         embedding = self.embedder(pixel_values)
    8:         embedding = self.pooler(embedding)
    9:         return embedding
   10: 


## 4. Testing with `dynamic=True` in ONNX Export

Let's see if ONNX export with `dynamic_axes` helps with the tensor shape issue.

In [5]:
import os
import tempfile

print("=== Testing ONNX Export with dynamic=True ===")

# Test 1: Pure ResNet with ONNX dynamic export
print("\n1. Pure ResNet + ONNX dynamic export:")
try:
    with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as tmp:
        torch.onnx.export(
            simple_resnet,
            dummy_input,
            tmp.name,
            dynamic_axes={
                'input': {0: 'batch_size', 2: 'height', 3: 'width'},
                'output': {0: 'batch_size'}
            },
            input_names=['input'],
            output_names=['output']
        )
        print("✅ Pure ResNet ONNX export with dynamic=True: SUCCESS")
        os.unlink(tmp.name)
except Exception as e:
    print(f"❌ Pure ResNet ONNX export failed: {e}")

# Test 2: HuggingFace ResNet with ONNX dynamic export
print("\n2. HuggingFace ResNet + ONNX dynamic export:")
try:
    with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as tmp:
        torch.onnx.export(
            hf_resnet,
            dummy_input,
            tmp.name,
            dynamic_axes={
                'pixel_values': {0: 'batch_size', 2: 'height', 3: 'width'},
                'logits': {0: 'batch_size'}
            },
            input_names=['pixel_values'],
            output_names=['logits']
        )
        print("✅ HuggingFace ResNet ONNX export with dynamic=True: SUCCESS")
        print("   Note: ONNX export works because it uses actual tensor values, not symbolic tracing")
        os.unlink(tmp.name)
except Exception as e:
    print(f"❌ HuggingFace ResNet ONNX export failed: {e}")

=== Testing ONNX Export with dynamic=True ===

1. Pure ResNet + ONNX dynamic export:
✅ Pure ResNet ONNX export with dynamic=True: SUCCESS

2. HuggingFace ResNet + ONNX dynamic export:


/mnt/d/BYOM/modelexport/.venv/lib/python3.12/site-packages/transformers/models/resnet/modeling_resnet.py:74: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


✅ HuggingFace ResNet ONNX export with dynamic=True: SUCCESS
   Note: ONNX export works because it uses actual tensor values, not symbolic tracing


## 5. Why FX Fails but ONNX Export Works

In [6]:
print("=== Understanding the Difference ===")
print()
print("🔍 FX Symbolic Tracing:")
print("   • Creates symbolic 'Proxy' objects instead of real tensors")
print("   • pixel_values.shape[1] returns a Proxy, not an int")
print("   • Proxy != int comparison cannot be resolved at trace time")
print("   • Result: 'symbolically traced variables cannot be used as inputs to control flow'")
print()
print("✅ ONNX Export:")
print("   • Uses actual tensor values during export")
print("   • pixel_values.shape[1] returns actual int (e.g., 3)")
print("   • 3 != 3 resolves to False, validation passes")
print("   • Result: Export succeeds")
print()
print("💡 Key Insight:")
print("   FX fails during SYMBOLIC tracing (before running)")
print("   ONNX works during ACTUAL execution (with real tensors)")
print("   dynamic=True doesn't help FX because the issue is at trace time, not runtime")

=== Understanding the Difference ===

🔍 FX Symbolic Tracing:
   • Creates symbolic 'Proxy' objects instead of real tensors
   • pixel_values.shape[1] returns a Proxy, not an int
   • Proxy != int comparison cannot be resolved at trace time
   • Result: 'symbolically traced variables cannot be used as inputs to control flow'

✅ ONNX Export:
   • Uses actual tensor values during export
   • pixel_values.shape[1] returns actual int (e.g., 3)
   • 3 != 3 resolves to False, validation passes
   • Result: Export succeeds

💡 Key Insight:
   FX fails during SYMBOLIC tracing (before running)
   ONNX works during ACTUAL execution (with real tensors)
   dynamic=True doesn't help FX because the issue is at trace time, not runtime


## 6. Demonstrating the Proxy Issue

In [7]:
print("=== Demonstrating Proxy vs Real Tensor ===")

# Create a minimal example that shows the proxy issue
class ProxyDemoModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.expected_channels = 3
        
    def forward(self, x):
        print(f"During execution - x.shape[1]: {x.shape[1]} (type: {type(x.shape[1])})")
        if x.shape[1] != self.expected_channels:
            raise ValueError(f"Expected {self.expected_channels} channels, got {x.shape[1]}")
        return x.sum()

demo_model = ProxyDemoModel()
demo_input = torch.randn(1, 3, 32, 32)

print("\n1. Normal execution:")
result = demo_model(demo_input)
print(f"   Result: {result.item():.2f}")

print("\n2. FX symbolic tracing:")
try:
    traced = torch.fx.symbolic_trace(demo_model)
    print("   ✅ FX tracing succeeded (unexpected!)")
except Exception as e:
    print(f"   ❌ FX tracing failed: {e}")
    print("   📍 This is the same error HuggingFace ResNet encounters")

=== Demonstrating Proxy vs Real Tensor ===

1. Normal execution:
During execution - x.shape[1]: 3 (type: <class 'int'>)
   Result: -48.91

2. FX symbolic tracing:
During execution - x.shape[1]: Proxy(getitem) (type: <class 'torch.fx.proxy.Proxy'>)
   ❌ FX tracing failed: symbolically traced variables cannot be used as inputs to control flow
   📍 This is the same error HuggingFace ResNet encounters


## 7. Solutions and Workarounds

In [ ]:
print("=== Solutions for HuggingFace Models ===")
print()
print("❌ dynamic=True does NOT help with FX because:")
print("   • FX fails at trace time (symbolic), not runtime")
print("   • dynamic=True is for ONNX export flexibility, not FX tracing")
print()
print("✅ Working solutions:")
print("   1. Use HTP strategy for HuggingFace models (as implemented)")
print("   2. Use pure PyTorch ResNet implementations for FX")
print("   3. Patch HuggingFace models to remove input validation")
print("   4. Use hybrid approach: auto-detect and fallback")
print()
print("🎯 Our Implementation Strategy:")
print("   • FX for pure PyTorch models (96.4% coverage on ProductionResNet)")
print("   • HTP for HuggingFace transformers models")
print("   • Automatic architecture detection and strategy selection")

## Conclusion

**Key Findings:**

1. **Pure ResNet works perfectly with FX** (96.4% coverage achieved)
2. **HuggingFace ResNet fails due to input validation** that compares tensor shapes
3. **`dynamic=True` does NOT help** because the issue is at symbolic trace time, not runtime
4. **ONNX export still works** because it uses real tensors, not symbolic proxies
5. **The solution is strategy selection**: FX for pure PyTorch, HTP for HuggingFace models

This demonstrates why our hybrid approach is valuable - it automatically detects model compatibility and chooses the appropriate strategy.